### Must run step by step as there are command lines to be executed in the middle of the flow

### Input files: Price-Today.csv
### Output files: daily-sales.csv, daily-sales-prices.csv

In [3]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
print(today)

2025-08-19


In [5]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'Today: {today}')
print(f'Yesterday: {yesterday}')

Today: 2025-08-19
Yesterday: 2025-08-18


In [7]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

In [9]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [11]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path : {dat_path}") 
print(f"Google Drive path : {god_path}")
print(f"iCloudDrive path : {icd_path}") 
print(f"OSD path : {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path : C:\Users\PC1\OneDrive\A5\Data
Google Drive path : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path : C:\Users\PC1\iCloudDrive\Data
OSD path : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [13]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.shape


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2025-08-18'
ORDER BY a.name



(16, 1)

In [15]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-08-18' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [18]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
#df.style.format(format_dict)
df

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,AIMIRT,2025-08-15,2025-08-18,9.85,9.75,360420,9.95,9.75,B,SET,-0.10,-1.02
1,CPF,2025-08-18,2025-08-18,24.40,23.80,61805413,24.40,23.60,B,SET50,-0.60,-2.46
2,DIF,2025-08-14,2025-08-18,8.40,8.15,73713998,8.45,8.15,B,SET,-0.25,-2.98
3,GVREIT,2025-08-15,2025-08-18,6.50,6.45,2953600,6.60,6.45,B,SET,-0.05,-0.77
4,JMART,2025-08-15,2025-08-18,8.25,8.15,41997182,8.45,8.05,B,SET50,-0.10,-1.21
5,JMT,2025-08-07,2025-08-18,12.10,11.40,301428468,12.60,11.10,B,SET50,-0.70,-5.79
6,PTT,2025-08-14,2025-08-18,32.25,32.25,274807978,32.50,32.00,B,SET50,0.00,0.00
7,RCL,2025-08-13,2025-08-18,30.25,27.50,29988394,31.00,27.50,B,SET100,-2.75,-9.09
8,SENA,2025-08-15,2025-08-18,1.79,1.83,458798,1.83,1.77,B,SET,0.04,2.23
9,TFFIF,2025-08-15,2025-08-18,6.10,6.05,5945160,6.10,6.00,B,SET,-0.05,-0.82


In [20]:
df.shape

(16, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [24]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,AIMIRT,2025-08-15,2025-08-18,9.85,9.75,"360,420",9.95,9.75,-1.02%,B
1,CPF,2025-08-18,2025-08-18,24.40,23.80,"61,805,413",24.40,23.60,-2.46%,B
2,DIF,2025-08-14,2025-08-18,8.40,8.15,"73,713,998",8.45,8.15,-2.98%,B
3,GVREIT,2025-08-15,2025-08-18,6.50,6.45,"2,953,600",6.60,6.45,-0.77%,B
4,JMART,2025-08-15,2025-08-18,8.25,8.15,"41,997,182",8.45,8.05,-1.21%,B
5,JMT,2025-08-07,2025-08-18,12.10,11.40,"301,428,468",12.60,11.10,-5.79%,B
6,PTT,2025-08-14,2025-08-18,32.25,32.25,"274,807,978",32.50,32.00,0.00%,B
7,RCL,2025-08-13,2025-08-18,30.25,27.50,"29,988,394",31.00,27.50,-9.09%,B
8,SENA,2025-08-15,2025-08-18,1.79,1.83,"458,798",1.83,1.77,2.23%,B
9,TFFIF,2025-08-15,2025-08-18,6.10,6.05,"5,945,160",6.10,6.00,-0.82%,B


In [26]:
file_name = "daily-sales.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [28]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\daily-sales.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales.csv


In [30]:
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(god_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(icd_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [32]:
sales = df[cols]
sales.shape

(16, 10)

In [34]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [36]:
prices = pd.read_csv(input_file)
prices.shape

(169, 10)

In [38]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge.shape

(16, 19)

In [40]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,AIMIRT,2025-08-15,2025-08-18,9.85,9.75,360420,9.95,9.75,-1.02,B,9.55,-0.20,304234,2025-08-19
1,CPF,2025-08-18,2025-08-18,24.40,23.80,61805413,24.40,23.60,-2.46,B,23.50,-0.30,28557794,2025-08-19
2,DIF,2025-08-14,2025-08-18,8.40,8.15,73713998,8.45,8.15,-2.98,B,8.10,-0.05,5094850,2025-08-19
3,GVREIT,2025-08-15,2025-08-18,6.50,6.45,2953600,6.60,6.45,-0.77,B,6.30,-0.15,3004285,2025-08-19
4,JMART,2025-08-15,2025-08-18,8.25,8.15,41997182,8.45,8.05,-1.21,B,8.25,0.10,22189989,2025-08-19
5,JMT,2025-08-07,2025-08-18,12.10,11.40,301428468,12.60,11.10,-5.79,B,11.20,-0.20,19913136,2025-08-19
6,PTT,2025-08-14,2025-08-18,32.25,32.25,274807978,32.50,32.00,0.00,B,31.75,-0.50,38972047,2025-08-19
7,RCL,2025-08-13,2025-08-18,30.25,27.50,29988394,31.00,27.50,-9.09,B,27.00,-0.50,3244741,2025-08-19
8,SENA,2025-08-15,2025-08-18,1.79,1.83,458798,1.83,1.77,2.23,B,1.83,0.00,203181,2025-08-19
9,TFFIF,2025-08-15,2025-08-18,6.10,6.05,5945160,6.10,6.00,-0.82,B,6.05,0.00,2164229,2025-08-19


In [42]:
file_name = "daily-sales-prices.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [44]:
print(f"Output file: {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file: C:\Users\PC1\OneDrive\A5\Data\daily-sales-prices.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales-prices.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales-prices.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales-prices.csv


In [46]:
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(god_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(icd_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

### Add or update sells record depends on trend

In [49]:
os.chdir(base_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [51]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
AIMIRT   2025-08-15 2025-08-19   9.85   9.55  -3.05%     360,420   9.95   9.55 B Update
CPF      2025-08-18 2025-08-19  24.40  23.50  -3.69%  61,805,413  24.40  23.50 B Update
DIF      2025-08-14 2025-08-19   8.40   8.10  -3.57%  73,713,998   8.45   8.10 B Update
GVREIT   2025-08-15 2025-08-19   6.50   6.30  -3.08%   2,953,600   6.60   6.30 B Update
IVL      2025-08-14 2025-08-19  22.20  20.30  -8.56%  91,017,626  23.50  20.30 S Update
JMART    2025-08-19 2025-08-19   8.15   8.25   1.23%  41,997,182   8.45   8.05 B Insert
JMT      2025-08-07 2025-08-19  12.10  11.20  -7.44% 301,428,468  12.60  11.10 B Update
MCS      2025-08-19 2025-08-19   8.55   8.60   0.58%     612,838   8.65   8.55 S Insert
NER      2025-08-14 2025-08-19   4.52   4.44  -1.77%  26,256,530   4.60   4.44 S Update
PTG      2025-08-15 2025-08-19  

In [53]:
os.chdir(current_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'



### cd\ruby\port_lite


### rails runner db/ins_sales.rb



In [55]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % today
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-08-19' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



In [57]:
df = pd.read_sql(sql, conlite)
df.shape

(16, 10)

In [59]:
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,AIMIRT,2025-08-15,2025-08-19,9.85,9.55,"720,840",9.95,9.55,B,SET,-0.30,-3.05%
1,CPF,2025-08-18,2025-08-19,24.40,23.50,"123,610,826",24.40,23.50,B,SET50,-0.90,-3.69%
2,DIF,2025-08-14,2025-08-19,8.40,8.10,"147,427,996",8.45,8.10,B,SET,-0.30,-3.57%
3,GVREIT,2025-08-15,2025-08-19,6.50,6.30,"5,907,200",6.60,6.30,B,SET,-0.20,-3.08%
4,JMART,2025-08-19,2025-08-19,8.15,8.25,"41,997,182",8.45,8.05,B,SET50,0.10,1.23%
5,JMT,2025-08-07,2025-08-19,12.10,11.20,"602,856,936",12.60,11.10,B,SET50,-0.90,-7.44%
6,PTT,2025-08-14,2025-08-19,32.25,31.75,"549,615,956",32.50,31.75,B,SET50,-0.50,-1.55%
7,RCL,2025-08-13,2025-08-19,30.25,27.00,"59,976,788",31.00,27.00,B,SET100,-3.25,-10.74%
8,SENA,2025-08-15,2025-08-19,1.79,1.83,"917,596",1.83,1.77,B,SET,0.04,2.23%
9,TFFIF,2025-08-15,2025-08-19,6.10,6.05,"11,890,320",6.10,6.00,B,SET,-0.05,-0.82%


In [61]:
df_sort = df.sort_values(by=['percent'],ascending=[True]).copy()

In [63]:
df_sort.query('percent <= -5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
7,RCL,2025-08-13,2025-08-19,30.25,27.00,"59,976,788",31.00,27.00,B,SET100,-3.25,-10.74%
12,IVL,2025-08-14,2025-08-19,22.20,20.30,"182,035,252",23.50,20.30,S,SET50,-1.90,-8.56%
5,JMT,2025-08-07,2025-08-19,12.10,11.20,"602,856,936",12.60,11.10,B,SET50,-0.90,-7.44%


In [65]:
df_sort.query('percent >= 5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent


In [67]:
conlite.close()

In [69]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-08-19 20:59:25
